In [3]:
import folium, altair as alt, geopandas as gpd, fiona, pandas as pd, requests, json
alt.renderers.enable('jupyterlab')

RendererRegistry.enable('jupyterlab')

In [4]:
print(folium.__version__)

0.6.0+71.g5706269


In [6]:
income = pd.read_csv(r"https://raw.githubusercontent.com/pri-data/50-states/master/data/income-counties-states-national.csv", dtype={"fips":str})
income['income-2015']=pd.to_numeric(income['income-2015'], errors='coerce')

In [7]:
income.head()

,fips,county,state,income-2015,income-1989a,income-1989b,change
0,00000,US,US,55775.0,28906,53367.28102,4.316843
1,01000,Alabama,AL,44833.0,22202,40990.11877,8.571546
2,01001,Autauga County,AL,56580.0,26898,49660.04030,12.230399
3,01003,Baldwin County,AL,52387.0,24043,44389.03818,15.267074
4,01005,Barbour County,AL,31433.0,18673,34474.75398,-9.676945


In [8]:
states = gpd.GeoDataFrame.from_features(requests.get(r"https://raw.githubusercontent.com/jtbaker/data/master/geodata/usstates.geojson").json(), crs=fiona.crs.from_epsg(4326))

states.head()

,CENSUSAREA,GEO_ID,LSAD,NAME,STATE,geometry
0,30842.923,0400000US23,,Maine,23,"(POLYGON ((-67.619761 44.519754, -67.61541 44...."
1,7800.058,0400000US25,,Massachusetts,25,"(POLYGON ((-70.832044 41.606504, -70.823735 41..."
2,56538.901,0400000US26,,Michigan,26,"(POLYGON ((-88.684434 48.115785, -88.675628 48..."
3,145545.801,0400000US30,,Montana,30,"POLYGON ((-104.057698 44.997431, -104.250145 4..."
4,109781.180,0400000US32,,Nevada,32,"POLYGON ((-114.0506 37.000396, -114.049995 36...."


In [9]:
abbrs = pd.read_json(json.dumps(requests.get('https://gist.githubusercontent.com/tvpmb/4734703/raw/b54d03154c339ed3047c66fefcece4727dfc931a/US%2520State%2520List').json()))

abbrs.head(3)

,alpha-2,name
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona


In [10]:
statesmerge = states.merge(abbrs,how='left', left_on='NAME', right_on='name')
statesmerge['geometry']=statesmerge.geometry.simplify(.05)

statesmerge.head()

,CENSUSAREA,GEO_ID,LSAD,NAME,STATE,geometry,alpha-2,name
0,30842.923,0400000US23,,Maine,23,"(POLYGON ((-67.619761 44.519754, -67.582113000...",ME,Maine
1,7800.058,0400000US25,,Massachusetts,25,"(POLYGON ((-70.832044 41.606504, -70.823735 41...",MA,Massachusetts
2,56538.901,0400000US26,,Michigan,26,"(POLYGON ((-88.684434 48.115785, -88.418244 48...",MI,Michigan
3,145545.801,0400000US30,,Montana,30,"POLYGON ((-104.057698 44.997431, -111.055199 4...",MT,Montana
4,109781.180,0400000US32,,Nevada,32,"POLYGON ((-114.0506 37.000396, -114.046838 36....",NV,Nevada


In [11]:
basechart = alt.Chart(income, title='State Income by County').mark_bar().encode(
    x=alt.X('county:N', title='County Name', sort=alt.EncodingSortField(op='sum', field='change', order='descending')), 
    y=alt.Y('change:Q', title='Change (%)'), 
    color=alt.Color('change:Q', scale=alt.Scale(domain=[-20,0,20], range=['red', 'grey', 'green']), legend=alt.Legend(orient=alt.LegendOrient('top-right'))),
    tooltip=[alt.Tooltip('county:N',title='County'),
             alt.Tooltip('state:N', title='State'), 
             alt.Tooltip('change:Q', format=".2f", title='Change'), 
             alt.Tooltip('income-2015:Q', format="$,.2f", title='Income - 2015')]
).properties(width=500)

basechart

<VegaLite 2 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [12]:
m = folium.Map(location=[35.3,-97.6], zoom_start=4)

g = folium.GeoJson(statesmerge, style_function=lambda x: {'fillColor':"transparent", "color":'black'}, 
                   tooltip=folium.features.GeoJsonTooltip(fields=["NAME"])).add_to(m)

pop = folium.features.GeoJsonPopup(fields=["NAME"],
                                   labels=False,
                                   vegalite=folium.VegaLite(basechart), 
                                   map_key='alpha-2',
                                   data_key='state',
                                   max_width=800,
                                   style='font-size: 20pt; margin: auto;',
                                   class_name='foliumPopup'
                                  )
g.add_child(pop)

m



<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><iframe src="data:text/html;charset=utf-8;base64,PCFET0NUWVBFIGh0bWw+CjxoZWFkPiAgICAKICAgIDxtZXRhIGh0dHAtZXF1aXY9ImNvbnRlbnQtdHlwZSIgY29udGVudD0idGV4dC9odG1sOyBjaGFyc2V0PVVURi04IiAvPgogICAgPHNjcmlwdD5MX1BSRUZFUl9DQU5WQVM9ZmFsc2U7IExfTk9fVE9VQ0g9ZmFsc2U7IExfRElTQUJMRV8zRD1mYWxzZTs8L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4zLjQvZGlzdC9sZWFmbGV0LmpzIj48L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2FqYXguZ29vZ2xlYXBpcy5jb20vYWpheC9saWJzL2pxdWVyeS8xLjExLjEvanF1ZXJ5Lm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvanMvYm9vdHN0cmFwLm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9jZG5qcy5jbG91ZGZsYXJlLmNvbS9hamF4L2xpYnMvTGVhZmxldC5hd2Vzb21lLW1hcmtlcnMvMi4wLjIvbGVhZmxldC5hd2Vzb21lLW1hcmtlcnMuanMiPjwvc2NyaXB0PgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4zLjQvZGlzdC9sZWFmbGV0LmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL21heGNkbi5ib290c3RyYXBjZG4uY29tL2Jvb3RzdHJhcC8zLjIuMC9jc3MvYm9vdHN0cmFwLm1pbi5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvY3NzL2Jvb3RzdHJhcC10aGVtZS5taW4uY3NzIi8+CiAgICA8bGluayByZWw9InN0eWxlc2hlZXQiIGhyZWY9Imh0dHBzOi8vbWF4Y2RuLmJvb3RzdHJhcGNkbi5jb20vZm9udC1hd2Vzb21lLzQuNi4zL2Nzcy9mb250LWF3ZXNvbWUubWluLmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2NkbmpzLmNsb3VkZmxhcmUuY29tL2FqYXgvbGlicy9MZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy8yLjAuMi9sZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9yYXdjZG4uZ2l0aGFjay5jb20vcHl0aG9uLXZpc3VhbGl6YXRpb24vZm9saXVtL21hc3Rlci9mb2xpdW0vdGVtcGxhdGVzL2xlYWZsZXQuYXdlc29tZS5yb3RhdGUuY3NzIi8+CiAgICA8c3R5bGU+aHRtbCwgYm9keSB7d2lkdGg6IDEwMCU7aGVpZ2h0OiAxMDAlO21hcmdpbjogMDtwYWRkaW5nOiAwO308L3N0eWxlPgogICAgPHN0eWxlPiNtYXAge3Bvc2l0aW9uOmFic29sdXRlO3RvcDowO2JvdHRvbTowO3JpZ2h0OjA7bGVmdDowO308L3N0eWxlPgogICAgCiAgICA8bWV0YSBuYW1lPSJ2aWV3cG9ydCIgY29udGVudD0id2lkdGg9ZGV2aWNlLXdpZHRoLAogICAgICAgIGluaXRpYWwtc2NhbGU9MS4wLCBtYXhpbXVtLXNjYWxlPTEuMCwgdXNlci1zY2FsYWJsZT1ubyIgLz4KICAgIDxzdHlsZT4jbWFwX2ZmODZiN2VmNjdmMjRjY2M4MDUyMDhkZTliYTg3ZWRiIHsKICAgICAgICBwb3NpdGlvbjogcmVsYXRpdmU7CiAgICAgICAgd2lkdGg6IDEwMC4wJTsKICAgICAgICBoZWlnaHQ6IDEwMC4wJTsKICAgICAgICBsZWZ0OiAwLjAlOwogICAgICAgIHRvcDogMC4wJTsKICAgICAgICB9CiAgICA8L3N0eWxlPgogICAgPHNjcmlwdCBzcmM9Imh0dHBzOi8vY2RuLmpzZGVsaXZyLm5ldC9ucG0vdmVnYUAzIj48L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL3ZlZ2EtbGl0ZUAyIj48L3NjcmlwdD4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9jZG5qcy5jbG91ZGZsYXJlLmNvbS9hamF4L2xpYnMvdmVnYS1lbWJlZC8zLjIwLjAvdmVnYS1lbWJlZC5jc3MiLz4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2NkbmpzLmNsb3VkZmxhcmUuY29tL2FqYXgvbGlicy92ZWdhLWVtYmVkLzMuMjAuMC92ZWdhLWVtYmVkLmpzIj48L3NjcmlwdD4KICAgIAogICAgICAgIDxzdHlsZT4KICAgICAgICAuZm9saXVtUG9wdXB7CiAgICAgICAgICAgIGZvbnQtc2l6ZTogMjBwdDsgbWFyZ2luOiBhdXRvOwogICAgICAgIH0KICAgICAgICA8L3N0eWxlPgogICAgICAgIAo8L2hlYWQ+Cjxib2R5PiAgICAKICAgIAogICAgPGRpdiBjbGFzcz0iZm9saXVtLW1hcCIgaWQ9Im1hcF9mZjg2YjdlZjY3ZjI0Y2NjODA1MjA4ZGU5YmE4N2VkYiIgPjwvZGl2Pgo8L2JvZHk+CjxzY3JpcHQ+ICAgIAogICAgCiAgICAKICAgICAgICB2YXIgYm91bmRzID0gbnVsbDsKICAgIAoKICAgIHZhciBtYXBfZmY4NmI3ZWY2N2YyNGNjYzgwNTIwOGRlOWJhODdlZGIgPSBMLm1hcCgKICAgICAgICAnbWFwX2ZmODZiN2VmNjdmMjRjY2M4MDUyMDhkZTliYTg3ZWRiJywgewogICAgICAgIGNlbnRlcjogWzM1LjMsIC05Ny42XSwKICAgICAgICB6b29tOiA0LAogICAgICAgIG1heEJvdW5kczogYm91bmRzLAogICAgICAgIGxheWVyczogW10sCiAgICAgICAgd29ybGRDb3B5SnVtcDogZmFsc2UsCiAgICAgICAgY3JzOiBMLkNSUy5FUFNHMzg1NywKICAgICAgICB6b29tQ29udHJvbDogdHJ1ZSwKICAgICAgICB9KTsKCiAgICAKICAgIAogICAgdmFyIHRpbGVfbGF5ZXJfNzM2ZTY3YjBhNzQ1NDNiMmIwNzEyZjg3YThlNDM5MTQgPSBMLnRpbGVMYXllcigKICAgICAgICAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLAogICAgICAgIHsKICAgICAgICAiYXR0cmlidXRpb24iOiBudWxsLAogICAgICAgICJkZXRlY3RSZXRpbmEiOiBmYWxzZSwKICA